<a href="https://colab.research.google.com/github/Jacob-McM/big-data/blob/main/instrument_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 2s (110 kB/s)
Reading package lis

In [7]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 04:02:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.76MB/s    in 0.2s    

2022-11-30 04:02:34 (5.76 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [9]:
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
instrument_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

instrument_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [10]:
## Record Count
instrument_reviews_df.count()

904765

In [11]:
instrument_na_df = instrument_reviews_df.dropna(how='any')
instrument_na_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [12]:
instrument_na_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

KEEP:

|review_id|customer_id|product_id|product_parent|review_date|product_title|customer_count|star_rating |helpful_votes|total_votes|vine|
|---|---|---|---|---|---|---|---|---|---|---|


DROP:

|marketplace|product_category|verified_purchace|review_headline|review_body|
|-|-|-|-|-|


In [13]:
instrument_database_df = instrument_na_df.drop('marketplace','product_category','verified_purchase','review_headline','review_body')
instrument_database_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|          3|            0|          1|   N| 2015-08-31|
|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|          5|            0|          0|   N| 2015-08-31|
|    6111003| RIZR67JKUDBI0|B0006VMBHI|     603261968|AudioQuest LP rec...|          3|            0|          1|   N| 2015-08-31|
|    1546619|R27HL570VNL85F|B002B55TRG|     575084461|Hohner Inc. 560BX...|          5|            0|          0|   N| 2015-08-31|
|   12222213|R34EBU9QDWJ1GD|B00N1YPXW2|     165236328|Blue Yeti USB Mic...|        

In [14]:
instrument_count = instrument_database_df.groupby("customer_id").agg({"customer_id":"count"})
instrument_count.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   27314089|                 1|
|    6711334|                 1|
|   44525866|                 4|
|   47461997|                 1|
|   42560427|                 1|
|   35225823|                 1|
|   52526865|                20|
|    1954060|                 1|
|   34202730|                 3|
|   40014361|                 1|
|   42719693|                 1|
|   27252006|                 1|
|   48297144|                 1|
|   12204397|                 1|
|   17090175|                 2|
|   13352125|                 9|
|   46958825|                28|
|     134138|                 1|
|     283456|                 1|
|   42847010|                 2|
+-----------+------------------+
only showing top 20 rows



In [16]:
instrument_join_df = instrument_database_df.join(instrument_count, on="customer_id", how='inner')
instrument_join_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+------------------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|count(customer_id)|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+------------------+
|      10136|  RQ7OXCB6FQGZ|B002GYPS3M|     889395489|Audio-Technica AT...|          5|            0|          0|   N| 2013-02-28|                 1|
|      10271| RJHNQQD3NAATU|B0056I7VYY|     405110012|Kala MK-SD-LBLBUR...|          5|            1|          2|   N| 2012-11-30|                 1|
|      10626| RCF3DB6OQEUXF|B0002F74E8|     647820337|Sony MDR7502 Prof...|          5|            1|          1|   N| 2014-11-05|                 1|
|      10637|R35CSJBJF4ENZ6|B000NRQSKQ|     198522889|D'Addario Phospho...|          5|            0

In [17]:
instrument_join_df = instrument_join_df.withColumnRenamed("count(customer_id)","customer_count")
instrument_join_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+--------------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|customer_count|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+--------------+
|      10136|  RQ7OXCB6FQGZ|B002GYPS3M|     889395489|Audio-Technica AT...|          5|            0|          0|   N| 2013-02-28|             1|
|      10271| RJHNQQD3NAATU|B0056I7VYY|     405110012|Kala MK-SD-LBLBUR...|          5|            1|          2|   N| 2012-11-30|             1|
|      10626| RCF3DB6OQEUXF|B0002F74E8|     647820337|Sony MDR7502 Prof...|          5|            1|          1|   N| 2014-11-05|             1|
|      10637|R35CSJBJF4ENZ6|B000NRQSKQ|     198522889|D'Addario Phospho...|          5|            0|          0|   N| 2013-

In [18]:
# Load a function that allows us to select columns
from pyspark.sql.functions import col

review_id_df = instrument_join_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RMDCHWD0Y5OZ9|   45610553|B00HH62VB6|     618218723| 2015-08-31|
| RZSL0BALIYUNU|   14640079|B003LRN53I|     986692292| 2015-08-31|
| RIZR67JKUDBI0|    6111003|B0006VMBHI|     603261968| 2015-08-31|
|R27HL570VNL85F|    1546619|B002B55TRG|     575084461| 2015-08-31|
|R34EBU9QDWJ1GD|   12222213|B00N1YPXW2|     165236328| 2015-08-31|
|R1WCUI4Z1SIQEO|   46018513|B001N4GRGS|     134151483| 2015-08-31|
| RL5LNO26GAVJ1|   10225065|B009PJRMHQ|     694166585| 2015-08-31|
|R3GYQ5W8JHP8SB|    6356995|B00NKBDAZS|     446431775| 2015-08-31|
|R30SHYQXGG5EYC|   35297198|B006MIU7U2|     125871705| 2015-08-31|
|R14YLXA56NP51I|   32139520|B000FIBD0I|     771888534| 2015-08-31|
|R1ZH0HSH38IOTZ|   36060782|B0002E52GG|      68535945| 2015-08-31|
|R3H53KLLC210XI|    5301309|B00RZIH52G|     725541773| 2015-08

In [19]:
products_df = instrument_join_df.select(['product_id','product_title'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00HH62VB6|AGPtek® 10 Isolat...|
|B003LRN53I|Sennheiser HD203 ...|
|B0006VMBHI|AudioQuest LP rec...|
|B002B55TRG|Hohner Inc. 560BX...|
|B00N1YPXW2|Blue Yeti USB Mic...|
|B001N4GRGS|Middle Atlantic P...|
|B009PJRMHQ|Kmise 1pc Pickgua...|
|B00NKBDAZS|Kealoha Concert U...|
|B006MIU7U2|Halco 80000 - MR1...|
|B000FIBD0I|Gator GPTBLACK Pl...|
|B0002E52GG|Hetman 1 - Light ...|
|B00RZIH52G|Dragonpad pop fil...|
|B001792BAU|DharmaObjects Rel...|
|B009GSKW1Y|Musiclily SSS Pla...|
|B0002F4TKA|Vic Firth America...|
|B00K17YFBW|Guitar Stand for ...|
|B00EQ24HJS|Generic 3PLY Faux...|
|B00IBOYTUE|Audio 2000 6525 3...|
|B00FBRUSAE|Sawtooth ST-AMP-1...|
|B0113D2QUO|Upado Unlimited G...|
+----------+--------------------+
only showing top 20 rows



In [20]:
customer_df = instrument_join_df.select(['customer_id','customer_count'])
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   27314089|             1|
|    6711334|             1|
|   44525866|             4|
|   44525866|             4|
|   44525866|             4|
|   44525866|             4|
|   47461997|             1|
|   42560427|             1|
|   35225823|             1|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
|   52526865|            20|
+-----------+--------------+
only showing top 20 rows



In [21]:
vine_df = instrument_join_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|
| RZSL0BALIYUNU|          5|            0|          0|   N|
| RIZR67JKUDBI0|          3|            0|          1|   N|
|R27HL570VNL85F|          5|            0|          0|   N|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|
|R30SHYQXGG5EYC|          5|            0|          0|   N|
|R14YLXA56NP51I|          5|            1|          1|   N|
|R1ZH0HSH38IOTZ|          5|            0|          0|   N|
|R3H53KLLC210XI|          4|            0|          0|   N|
|R3OOR877NGA8JK|          3|            0|          0|   N|
|R1BY7WKOZ3KMH0|          2|            

In [22]:
## Values hidden for privacy

aws_endpoint = '###'
aws_port = '###'
aws_db = '###'
aws_username = '###'
aws_password = '###'

In [23]:
jdbc_url=f'jdbc:postgresql://{aws_endpoint}:{aws_port}/{aws_db}'

config = {"user": f'{aws_username}', 
          "password": f'{aws_password}', 
          "driver":"org.postgresql.Driver"}

mode = 'overwrite' 

In [24]:
review_id_df .write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [25]:
products_df .write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [26]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [27]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [49]:
vine_count = vine_df.groupBy('vine').agg({'Vine':'count'})
vine_count.show()

+----+-----------+
|vine|count(Vine)|
+----+-----------+
|   Y|       2287|
|   N|     902376|
+----+-----------+



In [46]:
vine_rating_y = vine_df.groupBy('star_rating','vine').agg({'Vine':'count'}).filter(col('vine') == 'Y').show()
vine_rating_y['percentage'] = vine_rating_y('count(Vine)') / 

+-----------+----+-----------+
|star_rating|vine|count(Vine)|
+-----------+----+-----------+
|          5|   Y|       1290|
|          3|   Y|        182|
|          4|   Y|        749|
|          2|   Y|         49|
|          1|   Y|         17|
+-----------+----+-----------+



In [47]:
vine_rating_n = vine_df.groupBy('star_rating','vine').agg({'Vine':'count'}).filter(col('vine') == 'N').show()

+-----------+----+-----------+
|star_rating|vine|count(Vine)|
+-----------+----+-----------+
|          4|   N|     157767|
|          1|   N|      66119|
|          3|   N|      66861|
|          5|   N|     571554|
|          2|   N|      40075|
+-----------+----+-----------+



I do not think there is a bias present. 56% of Vine users found in these reviews opted to give 5 star reviews, while 63% of Non-Vine users opted to give 5 star reviews. Further investigation can be done to determine if the 7% difference between the two groups is significant. 

On the other side of the ratings, Non-Vine users opted for 1 star reviews 7% of the time. Interestingly, 7% of Vine users also opted for 1 star reviews. 

Overall, I do not believe being a specific user group creates bias reviews. 


Certain behavior might want to be monitored from the Vine users group to determine how they choose their products, such as how products are already trending, and the population of the user types rating that kind of item.